In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
import xarray as xr

In [3]:
#Replace with file path for this directory on your machine
!pip install -e /Users/justinmaynard/Documents/GitHub/assetraMP/ 

Obtaining file:///Users/justinmaynard/Documents/GitHub/assetraMP
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for assetra (pyproject.toml) ... done
  Created wheel for assetra: filename=assetra-1.0.3-py3-none-any.whl size=2689 sha256=dbec1acd386afa31ce06d71424f177f8781dc8432d27f26b9e6f1bc595d1c07a
  Stored in directory: /private/var/folders/zd/3zqsbnbn5lg29wjyl0df33l80000gn/T/pip-ephem-wheel-cache-xykrn5ps/wheels/c0/f5/c9/3b09bf7f2c6beadc5d6cde6ca7e50094804a4b90493c211e80
Successfully built assetra
  Attempting uninstall: assetra
    Found existing installation: assetra 1.0.3
    Uninstalling assetra-1.0.3:
      Successfully uninstalled assetra-1.0.3


In [4]:
p = Path(".")
root_folder = p.cwd().parent
root_folder
data_folder = root_folder / 'pjm_2023_test' / 'pjm_data'
data_folder
scripts_foler = root_folder/ 'pjm_2023_test'

### Demand Data

In [83]:
#Function to load demand data
def load_pjm_cleaned_hourly_demand(
        pjm_demand_file: Path,
        start_hour: datetime,
        end_hour: datetime) -> xr.DataArray:
    """Return hourly demand data as formatted data array.
    To use this function, download cleaned demand data from:

    https://github.com/truggles/EIA_Cleaned_Hourly_Electricity_Demand_Data

    Args:
        eia_930_cleaned_demand_file (Path): Path to hourly demand file
        start_hour (datetime): First timestamp to include
        end_hour (datetime): Last timestamp to include (inclusive)

    Returns:
        xr.DataArray: Hourly demand array with time dimension and datetime coordinates.
    """
    # read demand file
    pjm_demand_df = pd.read_csv(
        pjm_demand_file,
        usecols=["datetime_beginning_ept", "mw"],
        index_col="datetime_beginning_ept",
        parse_dates=True,
    )

    # keep cleaned demand demand
    pjm_hourly_demand_pd = pjm_demand_df["mw"].loc[start_hour:end_hour]

    # convert to xr.DataArray
    pjm_hourly_demand = xr.DataArray(
        data=pjm_hourly_demand_pd.values,
        coords=dict(
            time=pjm_hourly_demand_pd.index.values
        )
    )
    return pjm_hourly_demand

In [85]:
def sum_by_datetime(file_path):
    # Load the CSV file
    data = pd.read_csv(file_path)
    # Ensure datetime column is parsed correctly
    data['datetime_beginning_ept'] = pd.to_datetime(data['datetime_beginning_ept'])
    # Group by 'datetime_beginning_ept' and sum 'mw'
    grouped_data = data.groupby('datetime_beginning_ept', as_index=False)['mw'].sum()
    grouped_data.index = pd.to_datetime(grouped_data['datetime_beginning_ept'])
    grouped_data = grouped_data.drop(columns='datetime_beginning_ept')

    return grouped_data


In [89]:
demand_path = str(data_folder/"hrl_load_metered.csv")
summed_data = sum_by_datetime(demand_path)
summed_data.to_csv(data_folder / "pjm_load_summed.csv")

/var/folders/zd/3zqsbnbn5lg29wjyl0df33l80000gn/T/ipykernel_47256/3796617890.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['datetime_beginning_ept'] = pd.to_datetime(data['datetime_beginning_ept'])


In [95]:
def fill_missing_values(file_path):
    # Load the CSV file
    data = pd.read_csv(file_path, parse_dates=['datetime_beginning_ept'])
    
    # Set the datetime column as the index
    data.set_index('datetime_beginning_ept', inplace=True)
    
    # Create a complete range of datetimes for all the hours in the range of the dataset
    full_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')
    
    # Reindex the dataframe to include all hours in the range
    data = data.reindex(full_range)
    
    # Fill missing values via interpolation
    data['mw'] = data['mw'].interpolate()
    
    # Reset the index to make datetime_beginning_ept a column again
    data.reset_index(inplace=True)
    
    # Rename the columns to match the original
    data.columns = ['datetime_beginning_ept', 'mw']
    

    # Ensure the output has the same column names as the original
    
    return data

# Example usage
filled_df = fill_missing_values(data_folder / "pjm_load_summed.csv")
filled_df.to_csv(data_folder / "pjm_load_summed_cleaned.csv")

/var/folders/zd/3zqsbnbn5lg29wjyl0df33l80000gn/T/ipykernel_47256/1465976239.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')


In [97]:
#Load demand data
pjm_cleaned_demand_file = Path( data_folder / "pjm_load_summed_cleaned.csv")
hourly_demand = load_pjm_cleaned_hourly_demand(
	pjm_cleaned_demand_file,
	start_hour="2023-01-01 00:00:00",
	end_hour="2023-12-31 23:00:00"
)

In [98]:
from assetra.system import EnergySystem
from assetra.system import EnergySystemBuilder

builder = EnergySystemBuilder()
unit_count = 0

In [99]:
# create demand unit
from assetra.units import DemandUnit

builder.add_unit(
    DemandUnit(
        id=unit_count,
        hourly_demand=hourly_demand
    )
)
unit_count += 1

### EIA 860 Data

In [33]:
def load_eia_860_plants(eia_860_plant_file: Path, bal_auth: str) -> pd.DataFrame:
    """Return a subset of the EIA 860 plant file for plants in a balancing authority

    Args:
        eia_860_plant_file (Path): Path to hourly demand file
        bal_auth (str): Balancing authority code as defined by EIA-860

    Returns:
        pd.DataFrame: Plant code-indexed dataframe with plant latitude and longitude
    """
        # read file
    eia_860_plant_df = pd.read_excel(
        data_folder/"egrid2023_data_rev1.xlsx",
        sheet_name="PLNT23",
        skiprows=1,
        usecols=[
            "ORISPL", #Plant code
            "LAT",
            "LON",
            "ISORTO", #Balancing authority code
            "ORISPL", #primary fuel category
            "NAMEPCAP", #nameplate capacity
            "PLHTIAN", #heat input MMBtu
            "PLNGENAN", #annual generation Mwh
        ],
        index_col="ORISPL",
    )
    # filter
    eia_860_plant_df = eia_860_plant_df[
        eia_860_plant_df["ISORTO"] == bal_auth
    ]
    return eia_860_plant_df




In [49]:

def load_eia_860_generators(
    eia_860_generator_file: Path,
    eia_860_plants: pd.DataFrame,
    additional_cols: list=[],
    tech_filter: list=[],
    invert_tech_filter: bool=False
    ) -> pd.DataFrame:
    """Return dataframe with generators" latitude, longitude, technology, and nameplate capacity"""
    # read file
    eia_860_generator_df = pd.read_excel(
        eia_860_generator_file,
        skiprows=1,
        usecols=[
            "Plant Code",
            "Technology",
            "Nameplate Capacity (MW)",
            "Status"
        ] + additional_cols,
    )
    eia_860_generator_df = eia_860_generator_df.join(eia_860_plant_df, on="Plant Code", how="left")

    Natural_Gas_FC = 3.36 #($/MMBtu) https://www.eia.gov/electricity/annual/html/epa_07_01.html
    Petroleum_FC = 15.89 #($/MMBtu) https://www.eia.gov/electricity/annual/html/epa_07_01.html
    Coal_FC = 2.51 #($/MMBtu) https://www.eia.gov/electricity/annual/html/epa_07_01.html
    Nuclear_FC = 9
    Peotroleum_Coke_FC = Petroleum_FC
    Landfill_Gas_FC =  4 #assumption
    Wood_Waste_Biomass_FC = 2 #https://www.epa.gov/sites/default/files/2015-07/documents/biomass_combined_heat_and_power_catalog_of_technologies_7._representative_biomass_chp_system_cost_and_performance_profiles.pdf
    Municipal_Solid_Waste_FC = 4
    Other_Gases_FC = 4

    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Natural Gas Fired Combustion Turbine", "MC"] = Natural_Gas_FC
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Petroleum Liquids", "MC"] = Petroleum_FC
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Natural Gas Steam Turbine", "MC"] = Natural_Gas_FC
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Conventional Steam Coal", "MC"] = Coal_FC
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Nuclear", "MC"] = Nuclear_FC
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Natural Gas Internal Combustion Engine", "MC"] = Natural_Gas_FC
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Natural Gas Fired Combined Cycle", "MC"] = Natural_Gas_FC
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Petroleum Coke", "MC"] = Petroleum_FC
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Landfill Gas", "MC"] = Landfill_Gas_FC
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Wood/Wood Waste Biomass", "MC"] = Wood_Waste_Biomass_FC
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Municipal Solid Waste", "MC"] = Municipal_Solid_Waste_FC
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Other Gases", "MC"] = Other_Gases_FC
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Other Waste Biomass", "MC"] = Wood_Waste_Biomass_FC
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "All Other", "MC"] = Other_Gases_FC
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Flywheels", "MC"] = 0
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Other Natural Gas", "MC"] = Natural_Gas_FC
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Onshore Wind Turbine", "MC"] = 0
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Solar Photovoltaic", "MC"] = 0
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Offshore Wind Turbine", "MC"] = 0
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Batteries", "MC"] = 0
    eia_860_generator_df.loc[eia_860_generator_df["Technology"] == "Hydroelectric Pumped Storage", "MC"] = 0

    # filter by plants
    eia_860_generator_df = eia_860_generator_df[
        eia_860_generator_df["Plant Code"].isin(eia_860_plants.index)
    ]

    # filter by technology
    if tech_filter:
        if invert_tech_filter:
            eia_860_generator_df = eia_860_generator_df[
                ~eia_860_generator_df["Technology"].isin(
                    tech_filter
                )
            ]
        else:
            eia_860_generator_df = eia_860_generator_df[
                eia_860_generator_df["Technology"].isin(
                    tech_filter
                )
            ]

    # filter by status
    eia_860_generator_df = eia_860_generator_df[
        eia_860_generator_df["Status"] == "OP"
    ]

    eia_860_generator_df["LAT"] = eia_860_generator_df["Plant Code"].map(lambda plant_code: eia_860_plants["LAT"][plant_code])
    eia_860_generator_df["LON"] = eia_860_generator_df["Plant Code"].map(lambda plant_code: eia_860_plants["LON"][plant_code])

    return eia_860_generator_df

In [ ]:
eia_860_plants = load_eia_860_plants(data_folder/"egrid2023_data_rev1.xlsx", "PJM")

# parse eia 860 generator types
EIA_860_NON_THERMAL_TECHNOLOGY = [
    "Onshore Wind Turbine",
    #"Conventional Hydroelectric",
    "Solar Photovoltaic",
    "Offshore Wind Turbine",
    "Batteries",
    "Hydroelectric Pumped Storage"
]

eia_860_generator_file = Path(data_folder / "3_1_Generator_Y2023.xlsx")
eia_860_wind_file = Path(data_folder / "3_2_Wind_Y2023.xlsx")
eia_860_solar_file = Path(data_folder / "3_3_Solar_Y2023.xlsx")
eia_860_storage_file = Path(data_folder / "3_4_Energy_Storage_Y2023.xlsx")
eia_860_thermal_generators = load_eia_860_generators(
    eia_860_generator_file, 
    eia_860_plants,
    tech_filter=EIA_860_NON_THERMAL_TECHNOLOGY,
    invert_tech_filter=True
)

In [50]:

eia_860_wind_generators = load_eia_860_generators(
    eia_860_wind_file,
    eia_860_plants
)
eia_860_solar_generators = load_eia_860_generators(
    eia_860_solar_file,
    eia_860_plants
)
eia_860_storage_generators = load_eia_860_generators(
    eia_860_storage_file,
    eia_860_plants,
    additional_cols=["Nameplate Energy Capacity (MWh)"]
)

### Import MERRA

In [102]:
import xarray as xr

# load processed power generation dataset (solar cf, wind cf, and temperature)
pow_gen_file = Path(data_folder / "pjm_power_generation_2023.nc")
pow_gen_dataset = xr.open_dataset(pow_gen_file)

def get_nearest_hourly_profile(
    latitude: float,
    longitude: float,
    array: xr.DataArray
) -> xr.DataArray:
    """Return time series corresponding to the nearest coordinate in a
    MERRA power generation data array.

    Args:
        latitude (float): Latitude relative to equator in degrees
        start_hour (datetime): Longitude relative to meridian in degrees
        array (xr.DataArray): "solar_capacity_factor", "wind_capacity_factor",
            or "temperature"

    Returns:
        xr.DataArray: Array with time dimension and datetime coordinates.
    """
    return array.sel(
            lat=latitude, 
            lon=longitude, 
            method="nearest"
        ).squeeze(drop=True)

def get_merra_power_generation_solar_cf(
    latitude: float,
    longitude: float) -> xr.DataArray:
    return get_nearest_hourly_profile(latitude, longitude, pow_gen_dataset["solar_capacity_factor"])

def get_merra_power_generation_wind_cf(
    latitude: float,
    longitude: float) -> xr.DataArray:
    return get_nearest_hourly_profile(latitude, longitude, pow_gen_dataset["wind_capacity_factor"])

def get_merra_power_generation_temperature(
    latitude: float,
    longitude: float) -> xr.DataArray:
    return get_nearest_hourly_profile(latitude, longitude, pow_gen_dataset["temperature"])

In [103]:
import pandas as pd

# load temperature dependent outage rate (tdfor) table
tdfor_table_file = Path(data_folder / "temperature_dependent_outage_rates.csv")
tdfor_table = pd.read_csv(tdfor_table_file, index_col=0)
tdfor_table = tdfor_table / 100 # percentages stored as integers

# create mapping table for tdfor table
tech_categories = {
    "CC" : ["Natural Gas Fired Combined Cycle"],
    "CT" : ["Natural Gas Fired Combustion Turbine","Landfill Gas"],
    "DS" : ["Natural Gas Internal Combustion Engine"],
    "ST" : ["Conventional Steam Coal","Natural Gas Steam Turbine"],
    "NU" : ["Nuclear"],
    "HD" : ["Conventional Hydroelectric","Solar Thermal without Energy Storage",
                   "Hydroelectric Pumped Storage","Solar Thermal with Energy Storage","Wood/Wood Waste Biomass"]
}

# create mapping from technology to category
tech_mapping = {tech : cat for cat, techs in tech_categories.items() for tech in techs}

def get_hourly_forced_outage_rate(hourly_temperature: xr.DataArray, technology: str) -> xr.DataArray:
    # index tdfor table by tech
    tdfor_map = tdfor_table[tech_mapping.get(technology, "Other")]
    map_temp_to_for = lambda hourly_temperature: tdfor_map.iloc[
            tdfor_map.index.get_indexer(hourly_temperature, method="nearest")
        ]
    return xr.apply_ufunc(
        map_temp_to_for,
        hourly_temperature
    ).rename("hourly_forced_outage_rate")


### Build Units

In [104]:
from assetra.units import StochasticUnit

for _, generator in eia_860_thermal_generators.iterrows():
    # get hourly temperature
    hourly_temperature = get_merra_power_generation_temperature(
        generator["Latitude"],
        generator["Longitude"]
    )

    # map temperature to hourly forced outage rate
    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])

    # get hourly capacity
    hourly_capacity = ( 
        xr.ones_like(hourly_temperature).rename("hourly_capacity") 
        * generator["Nameplate Capacity (MW)"]
    )

    # create assetra energy unit
    thermal_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator["Nameplate Capacity (MW)"],
            hourly_capacity=hourly_capacity,
            hourly_forced_outage_rate=hourly_forced_outage_rate
        )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(thermal_unit)

from assetra.units import StochasticUnit

# add solar
for _, generator in eia_860_solar_generators.iterrows():
    # get hourly temperature
    hourly_temperature = get_merra_power_generation_temperature(
        generator["Latitude"],
        generator["Longitude"]
    )
    # get hourly temperature
    hourly_capacity = get_merra_power_generation_solar_cf(
        generator["Latitude"],
        generator["Longitude"]
    ) * generator["Nameplate Capacity (MW)"]

    # map temperature to hourly forced outage rate
    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])

    # create assetra energy unit
    solar_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator["Nameplate Capacity (MW)"],
            hourly_capacity=hourly_capacity,
            hourly_forced_outage_rate=hourly_forced_outage_rate
        )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(solar_unit)

# add wind
for _, generator in eia_860_wind_generators.iterrows():
    # get hourly temperature
    hourly_temperature = get_merra_power_generation_temperature(
        generator["Latitude"],
        generator["Longitude"]
    )
    # get hourly temperature
    hourly_capacity = get_merra_power_generation_wind_cf(
        generator["Latitude"],
        generator["Longitude"]
    ) * generator["Nameplate Capacity (MW)"]

    # map temperature to hourly forced outage rate
    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])

    # create assetra energy unit
    wind_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator["Nameplate Capacity (MW)"],
            hourly_capacity=hourly_capacity,
            hourly_forced_outage_rate=hourly_forced_outage_rate
        )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(wind_unit)



In [105]:
from assetra.units import StorageUnit

STORAGE_EFFICIENCY = 0.8

for _, generator in eia_860_storage_generators.iterrows():
    storage_duration = generator["Nameplate Energy Capacity (MWh)"] / generator["Nameplate Capacity (MW)"]
    if storage_duration < 4:
        storage_class = '2'
    elif 4 <= storage_duration < 6:
        storage_class = '4'
    elif 6 <= storage_duration < 8:
        storage_class = '6'
    elif 8 <= storage_duration < 10:
        storage_class = '8'
    else:
        storage_class = '12'

    storage_unit = StorageUnit(
        id=unit_count,
        nameplate_capacity=generator["Nameplate Capacity (MW)"],
        charge_rate=generator["Nameplate Capacity (MW)"],
        discharge_rate=generator["Nameplate Capacity (MW)"],
        charge_capacity=generator["Nameplate Energy Capacity (MWh)"],
        roundtrip_efficiency = STORAGE_EFFICIENCY,
        storage_duration = generator["Nameplate Energy Capacity (MWh)"] / generator["Nameplate Capacity (MW)"],
        storage_class = storage_class       

        
    )
    unit_count += 1

    # add unit to energy system
    builder.add_unit(storage_unit)

In [106]:

pjm_system_dir = Path(data_folder / "pjm_energy_system")

if pjm_system_dir.exists():
    energy_system = EnergySystem()
    energy_system.load(pjm_system_dir)
else:
    print("PJM saved system not found. Please create and save this system following the instructions found in the appendix") 

PJM saved system not found. Please create and save this system following the instructions found in the appendix


In [107]:
energy_system = builder.build()
energy_system.save(pjm_system_dir)

In [108]:
if pjm_system_dir.exists():
    energy_system = EnergySystem()
    energy_system.load(pjm_system_dir)
    print("Energy system loaded successfully")
else:
    print("PJM saved system not found. Please create and save this system following the instructions found in the appendix") 

Energy system loaded successfully


In [109]:
print("# of Units:", energy_system.size)
print("Sys. Capacity (MW):", round(energy_system.system_capacity))

# of Units: 3466
Sys. Capacity (MW): 212271


In [112]:
df = xr.open_dataset(data_folder/ "pjm_energy_system/StochasticUnit.assetra.nc").to_dataframe()
df


nameplate_capacity  hourly_capacity  \
energy_unit time                                                       
1           2023-01-01 00:00:00               149.0       149.000000   
            2023-01-01 01:00:00               149.0       149.000000   
            2023-01-01 02:00:00               149.0       149.000000   
            2023-01-01 03:00:00               149.0       149.000000   
            2023-01-01 04:00:00               149.0       149.000000   
...                                             ...              ...   
3433        2023-12-31 19:00:00               108.3        38.192397   
            2023-12-31 20:00:00               108.3        39.486437   
            2023-12-31 21:00:00               108.3        36.138925   
            2023-12-31 22:00:00               108.3        54.417307   
            2023-12-31 23:00:00               108.3        68.164960   

                                 hourly_forced_outage_rate   lat     lon  
energy_unit time                                                          
1           2023-01-01 00:00:00                      0.024  38.0 -84.375  
            2023-01-01 01:00:00                      0.024  38.0 -84.375  
            2023-01-01 02:00:00                      0.024  38.0 -84.375  
            2023-01-01 03:00:00                      0.024  38.0 -84.375  
            2023-01-01 04:00:00                      0.024  38.0 -84.375  
...                                                    ...   ...     ...  
3433        2023-12-31 19:00:00                      0.050  38.0 -84.375  
            2023-12-31 20:00:00                      0.050  38.0 -84.375  
            2023-12-31 21:00:00                      0.050  38.0 -84.375  
            2023-12-31 22:00:00                      0.050  38.0 -84.375  
            2023-12-31 23:00:00                      0.050  38.0 -84.375  

[30073080 rows x 5 columns]

In [110]:
from assetra.simulation import ProbabilisticSimulation

simulation = ProbabilisticSimulation(
    start_hour="2023-01-01 00:00:00",
    end_hour="2023-12-31 23:00:00",
    trial_size=1
)

simulation.assign_energy_system(energy_system)
simulation.run()

The ProbabilisticSimulation object generates a net hourly capacity matrix, representing net system capacity in each Monte Carlo trial. We can access a copy of this matrix to analyze shortfalls.

In [52]:
# convert net hourly capacity matrix to pandas dataframe with risk hours only
shortfall_matrix_pd = simulation.net_hourly_capacity_matrix.where(lambda c: c < 0).to_pandas().T.dropna(how="all")
shortfall_matrix_pd

trial,0
time,
2023-01-05 19:00:00,-368.305916
2023-01-06 07:00:00,-2540.894064
2023-01-06 08:00:00,-5437.198444
2023-01-06 09:00:00,-13061.033069
2023-01-06 10:00:00,-9731.663815
...,...
2023-12-29 18:00:00,-3444.746055
2023-12-29 19:00:00,-1561.425087
2023-12-29 20:00:00,-2021.654969



We can calculate hourly loss of load probability from the net hourly capacity matrix

In [53]:
# get loss of load probability
loss_of_load_prob = shortfall_matrix_pd.count(axis=1) / shortfall_matrix_pd.shape[1]

# show top 10 risk hours
loss_of_load_prob.sort_values(ascending=False)[:10]

time
2023-01-05 19:00:00    1.0
2023-08-19 17:00:00    1.0
2023-08-18 07:00:00    1.0
2023-08-18 08:00:00    1.0
2023-08-18 09:00:00    1.0
2023-08-18 10:00:00    1.0
2023-08-18 11:00:00    1.0
2023-08-18 12:00:00    1.0
2023-08-18 13:00:00    1.0
2023-08-18 14:00:00    1.0
dtype: float64

In [54]:
# show shortfalls in first 5 trials
shortfall_matrix_pd.loc[:,:5]

trial,0
time,
2023-01-05 19:00:00,-368.305916
2023-01-06 07:00:00,-2540.894064
2023-01-06 08:00:00,-5437.198444
2023-01-06 09:00:00,-13061.033069
2023-01-06 10:00:00,-9731.663815
...,...
2023-12-29 18:00:00,-3444.746055
2023-12-29 19:00:00,-1561.425087
2023-12-29 20:00:00,-2021.654969


In [126]:
!pip install matplotlib

In [24]:
from assetra.metrics import ExpectedUnservedEnergy

# instantiate eue model
eue_model = ExpectedUnservedEnergy(simulation)
eue = eue_model.evaluate()

print("System EUE:", round(eue, 2), "MWh")

System EUE: 67480341.83 MWh


In [192]:
from assetra.metrics import LossOfLoadHours, LossOfLoadDays, LossOfLoadFrequency
import pandas as pd

adequacy = pd.Series(dtype=float)

for name, metric in [
    ("EUE (MWh)", ExpectedUnservedEnergy),
    ("LOLH (h)", LossOfLoadHours),
    ("LOLD (d)", LossOfLoadDays),
    ("LOLF (#)", LossOfLoadFrequency)
]:
    adequacy[name] = metric(simulation).evaluate()

# show results
adequacy.round(1)

EUE (MWh)    67383436.8
LOLH (h)         2610.0
LOLD (d)          226.0
LOLF (#)          279.0
dtype: float64

In [132]:
adequacy["Average Outage Duration (h)"] = adequacy["LOLH (h)"] / adequacy["LOLF (#)"]
adequacy["Average Shortfall (MW)"] = adequacy["EUE (MWh)"] / adequacy["LOLH (h)"]

# show results
adequacy.round(1)


EUE (MWh)                      67383436.8
LOLH (h)                           2610.0
LOLD (d)                            226.0
LOLF (#)                            279.0
Average Outage Duration (h)           9.4
Average Shortfall (MW)            25817.4
dtype: float64

Quantify resource contribution (ELCC)

Resource contribution is a typical extension of resource adequacy analysis. The assetra package implements effective load-carrying capability (ELCC) to quantify resource contribution. When we instantiate an EffectiveLoadCarryingCapability object, the base system will automatically be evaluated according to the ResourceAdequacyMetric type we provide. In the following example, we indicate that resource adequacy should be defined as EUE (e.g. rather than LOLH) by passing ExpectedUnservedEnergy (the class not an instance) as the last parameter to the ELCC instance.


In [134]:
from assetra.contribution import EffectiveLoadCarryingCapability
from assetra.simulation import ProbabilisticSimulation
from assetra.metrics import ExpectedUnservedEnergy

# initialize elcc model
elcc_model = EffectiveLoadCarryingCapability(
    energy_system,
    ProbabilisticSimulation(
        start_hour="2023-01-01 00:00:00",
        end_hour="2023-12-31 23:00:00",
        trial_size=100
    ),
    ExpectedUnservedEnergy
)


INFO:assetra.units:Using chunk size 115
INFO:assetra.units:Sampling outages for units 1-115 of 3433
INFO:assetra.units:Sampling outages for units 116-230 of 3433
INFO:assetra.units:Sampling outages for units 231-345 of 3433
INFO:assetra.units:Sampling outages for units 346-460 of 3433
INFO:assetra.units:Sampling outages for units 461-575 of 3433
INFO:assetra.units:Sampling outages for units 576-690 of 3433
INFO:assetra.units:Sampling outages for units 691-805 of 3433
INFO:assetra.units:Sampling outages for units 806-920 of 3433
INFO:assetra.units:Sampling outages for units 921-1035 of 3433
INFO:assetra.units:Sampling outages for units 1036-1150 of 3433
INFO:assetra.units:Sampling outages for units 1151-1265 of 3433
INFO:assetra.units:Sampling outages for units 1266-1380 of 3433
INFO:assetra.units:Sampling outages for units 1381-1495 of 3433
INFO:assetra.units:Sampling outages for units 1496-1610 of 3433
INFO:assetra.units:Sampling outages for units 1611-1725 of 3433
INFO:assetra.units:

In [138]:
storageUnits = xr.open_dataset(data_folder/ "pjm_energy_system/StorageUnit.assetra.nc")
storageUnits.to_dataframe() #.to_csv("demandunit.csv")

,nameplate_capacity,charge_rate,discharge_rate,charge_capacity,roundtrip_efficiency
energy_unit,,,,,
0,2.0,2.0,2.0,1.0,0.8
1,2.0,2.0,2.0,0.8,0.8
2,1.0,1.0,1.0,1.0,0.8
3,10.4,10.4,10.4,10.4,0.8
4,20.0,20.0,20.0,8.3,0.8
...,...,...,...,...,...
3493,12.0,12.0,12.0,12.0,0.8
3494,20.0,20.0,20.0,20.0,0.8
3495,7.0,7.0,7.0,7.0,0.8


In [162]:
eia_860_new_storage_generators = pd.DataFrame(columns = eia_860_storage_generators.columns)
eia_860_new_storage_generators.loc[0] = [9999, 'OP', 'Batteries', 10, 10, "", ""]

In [163]:
eia_860_new_storage_generators

,Plant Code,Status,Technology,Nameplate Capacity (MW),Nameplate Energy Capacity (MWh),Latitude,Longitude
0,9999,OP,Batteries,10,10,,


In [164]:
builder = EnergySystemBuilder()

for _, generator in eia_860_new_storage_generators.iterrows():
    storage_unit = StorageUnit(
        id=unit_count,
        nameplate_capacity=generator["Nameplate Capacity (MW)"],
        charge_rate=generator["Nameplate Capacity (MW)"],
        discharge_rate=generator["Nameplate Capacity (MW)"],
        charge_capacity=generator["Nameplate Energy Capacity (MWh)"],
        roundtrip_efficiency=STORAGE_EFFICIENCY
    )
    unit_count += 1
    builder.add_unit(storage_unit)

    # add unit to energy system
additional_system = builder.build()

In [175]:
eia_860_new_storage_generators.loc[0]['Nameplate Capacity (MW)']

10

In [166]:
elcc = elcc_model.evaluate(additional_system)
elcc_pct = elcc / eia_860_new_storage_generators.loc[0]['Nameplate Capacity (MW)'] * 100

INFO:assetra.units:Dispatching storage unit 0 of 1 in all hours
INFO:assetra.units:Dispatching storage unit 0 of 64 in all hours
INFO:assetra.units:Dispatching storage unit 1 of 64 in all hours
INFO:assetra.units:Dispatching storage unit 2 of 64 in all hours
INFO:assetra.units:Dispatching storage unit 3 of 64 in all hours
INFO:assetra.units:Dispatching storage unit 4 of 64 in all hours
INFO:assetra.units:Dispatching storage unit 5 of 64 in all hours
INFO:assetra.units:Dispatching storage unit 6 of 64 in all hours
INFO:assetra.units:Dispatching storage unit 7 of 64 in all hours
INFO:assetra.units:Dispatching storage unit 8 of 64 in all hours
INFO:assetra.units:Dispatching storage unit 9 of 64 in all hours
INFO:assetra.units:Dispatching storage unit 10 of 64 in all hours
INFO:assetra.units:Dispatching storage unit 11 of 64 in all hours
INFO:assetra.units:Dispatching storage unit 12 of 64 in all hours
INFO:assetra.units:Dispatching storage unit 13 of 64 in all hours
INFO:assetra.units:Dis

In [167]:
# display result
print("ELCC (%): ", round(elcc_pct, 1))



ELCC (%):  9.0
